In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.tree import DecisionTreeRegressor
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# BASE
# ------------------------------------------------------
import numpy as np
import pandas as pd
import os
import gc
import warnings

# PACF - ACF
# ------------------------------------------------------
import statsmodels.api as sm

# DATA VISUALIZATION
# ------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


# CONFIGURATIONS
# ------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings('ignore')

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv',parse_dates=['date'])
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv',parse_dates=['date'])
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv',parse_dates=['date'])

In [3]:
file1 = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv', parse_dates=['date'],infer_datetime_format= True)
file1.head(10)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0
1,1,2013-01-01,1,BABY CARE,0.00,0
2,2,2013-01-01,1,BEAUTY,0.00,0
3,3,2013-01-01,1,BEVERAGES,0.00,0
4,4,2013-01-01,1,BOOKS,0.00,0
5,5,2013-01-01,1,BREAD/BAKERY,0.00,0
6,6,2013-01-01,1,CELEBRATION,0.00,0
7,7,2013-01-01,1,CLEANING,0.00,0
8,8,2013-01-01,1,DAIRY,0.00,0
9,9,2013-01-01,1,DELI,0.00,0


In [4]:
file_test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv',
                    parse_dates=['date'],
                    infer_datetime_format=True)
file_test.head(1)

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0


In [5]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [7]:
# Interpolate missing values using linear interpolation
oil_interpolated = oil.interpolate(method='linear')

# Create two separate traces: one for original data and one for interpolated data
fig = px.line(oil, x='date', y='dcoilwtico', title="Daily Oil Price")
# fig.add_scatter(x=oil_interpolated['date'], y=oil_interpolated['dcoilwtico'], mode='lines', name='Interpolated', line=dict(color='red'))

# Show the plot
fig.show()

In [8]:
px.line(oil_interpolated, x='date', y='dcoilwtico', title="Daily Oil Price")

In [9]:
# Filling missing values using backward fill
oil.dcoilwtico = oil.dcoilwtico.bfill()

# Resampling and filling values with forward fill
oil = oil.set_index('date').resample('D').ffill().reset_index()

In [10]:
px.line(oil, x='date', y='dcoilwtico', title="Daily Oil Price")

In [11]:
# Labelling the locale column as numbers (starting from 1 since 0 = no holiday)
# expecting holidays play a role in sales made

holidays.locale = holidays.locale.map({'Regional':1,'Local':2,'National':3})
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,2,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,1,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,2,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,2,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,2,Riobamba,Cantonizacion de Riobamba,False


In [12]:
# checking how many duplicated entries are presents
holidays[holidays.date.duplicated(keep=False)]

,date,type,locale,locale_name,description,transferred
7,2012-06-25,Holiday,1,Imbabura,Provincializacion de Imbabura,False
8,2012-06-25,Holiday,2,Latacunga,Cantonizacion de Latacunga,False
9,2012-06-25,Holiday,2,Machala,Fundacion de Machala,False
10,2012-07-03,Holiday,2,Santo Domingo,Fundacion de Santo Domingo,False
11,2012-07-03,Holiday,2,El Carmen,Cantonizacion de El Carmen,False
...,...,...,...,...,...,...
319,2017-07-03,Holiday,2,Santo Domingo,Fundacion de Santo Domingo,False
341,2017-12-08,Holiday,2,Loja,Fundacion de Loja,False
342,2017-12-08,Transfer,2,Quito,Traslado Fundacion de Quito,False
344,2017-12-22,Holiday,2,Salinas,Cantonizacion de Salinas,False


In [13]:
holidays = holidays.groupby('date').max(numeric_only=True)
holidays.reset_index(inplace=True)

# Drop transferred column
holidays.drop('transferred', axis=1, inplace=True)

In [14]:
df = pd.merge(file1,oil,on='date',how='left')
df = pd.merge(df,stores,on='store_nbr',how='left')
df = pd.merge(df,holidays,on='date',how='left')

In [15]:
df.isnull().sum()

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
dcoilwtico           0
city                 0
state                0
type                 0
cluster              0
locale         2551824
dtype: int64

In [16]:
# fill no holidays with 0
df.locale.fillna(0,inplace=True)
df.isnull().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dcoilwtico     0
city           0
state          0
type           0
cluster        0
locale         0
dtype: int64

In [17]:
df.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,city,state,type,cluster,locale
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,93.14,Quito,Pichincha,D,13,3.00
1,1,2013-01-01,1,BABY CARE,0.00,0,93.14,Quito,Pichincha,D,13,3.00
2,2,2013-01-01,1,BEAUTY,0.00,0,93.14,Quito,Pichincha,D,13,3.00
3,3,2013-01-01,1,BEVERAGES,0.00,0,93.14,Quito,Pichincha,D,13,3.00
4,4,2013-01-01,1,BOOKS,0.00,0,93.14,Quito,Pichincha,D,13,3.00


In [18]:
# Repeating the procedure to test dateset
df_test = pd.merge(file_test,oil,on='date',how='left')
df_test = pd.merge(df_test,stores,on='store_nbr',how='left')
df_test = pd.merge(df_test,holidays,on='date',how='left')
df_test.locale.fillna(0,inplace=True)
df_test.isnull().sum()

id             0
date           0
store_nbr      0
family         0
onpromotion    0
dcoilwtico     0
city           0
state          0
type           0
cluster        0
locale         0
dtype: int64

In [19]:
df_test

,id,date,store_nbr,family,onpromotion,dcoilwtico,city,state,type,cluster,locale
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.80,Quito,Pichincha,D,13,0.00
1,3000889,2017-08-16,1,BABY CARE,0,46.80,Quito,Pichincha,D,13,0.00
2,3000890,2017-08-16,1,BEAUTY,2,46.80,Quito,Pichincha,D,13,0.00
3,3000891,2017-08-16,1,BEVERAGES,20,46.80,Quito,Pichincha,D,13,0.00
4,3000892,2017-08-16,1,BOOKS,0,46.80,Quito,Pichincha,D,13,0.00
...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,47.26,Quito,Pichincha,B,6,0.00
28508,3029396,2017-08-31,9,PREPARED FOODS,0,47.26,Quito,Pichincha,B,6,0.00
28509,3029397,2017-08-31,9,PRODUCE,1,47.26,Quito,Pichincha,B,6,0.00
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,Quito,Pichincha,B,6,0.00


In [20]:
sample = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')
sample.head(10)

,id,sales
0,3000888,0.00
1,3000889,0.00
2,3000890,0.00
3,3000891,0.00
4,3000892,0.00
5,3000893,0.00
6,3000894,0.00
7,3000895,0.00
8,3000896,0.00
9,3000897,0.00


In [21]:
df.drop(columns='onpromotion',inplace=True)
df_test.drop(columns='onpromotion',inplace=True)

In [22]:
df['day'] = df['date'].dt.weekday
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df_test['day'] = df_test['date'].dt.weekday
df_test['month'] = df_test['date'].dt.month
df_test['year'] = df_test['date'].dt.year

In [23]:
family_list = list(df['family'].unique())
df['family_int'] = [family_list.index(x) for x in df['family']]
df_test['family_int'] = [family_list.index(x) for x in df_test['family']]
df.drop(columns = ['family'], inplace = True)
df_test.drop(columns = ['family'], inplace = True)

In [24]:
df.drop(columns = ['city','state','type','cluster'], inplace = True)
df_test.drop(columns = ['city','state','type','cluster'], inplace = True)

In [25]:
df['store_nbr'].unique()

array([ 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  2, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29,  3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,  4,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49,  5, 50, 51, 52, 53, 54,  6,
        7,  8,  9])

In [26]:
df_test.head(10)

,id,date,store_nbr,dcoilwtico,locale,day,month,year,family_int
0,3000888,2017-08-16,1,46.80,0.00,2,8,2017,0
1,3000889,2017-08-16,1,46.80,0.00,2,8,2017,1
2,3000890,2017-08-16,1,46.80,0.00,2,8,2017,2
3,3000891,2017-08-16,1,46.80,0.00,2,8,2017,3
4,3000892,2017-08-16,1,46.80,0.00,2,8,2017,4
5,3000893,2017-08-16,1,46.80,0.00,2,8,2017,5
6,3000894,2017-08-16,1,46.80,0.00,2,8,2017,6
7,3000895,2017-08-16,1,46.80,0.00,2,8,2017,7
8,3000896,2017-08-16,1,46.80,0.00,2,8,2017,8
9,3000897,2017-08-16,1,46.80,0.00,2,8,2017,9


In [27]:
xdf_train = df.drop(columns=['sales','id'])
ydf_train = df['sales']
xdf_train.head()

,date,store_nbr,dcoilwtico,locale,day,month,year,family_int
0,2013-01-01,1,93.14,3.00,1,1,2013,0
1,2013-01-01,1,93.14,3.00,1,1,2013,1
2,2013-01-01,1,93.14,3.00,1,1,2013,2
3,2013-01-01,1,93.14,3.00,1,1,2013,3
4,2013-01-01,1,93.14,3.00,1,1,2013,4


In [28]:
xdf_test = df_test.drop(columns = ['id'])
xdf_test 

,date,store_nbr,dcoilwtico,locale,day,month,year,family_int
0,2017-08-16,1,46.80,0.00,2,8,2017,0
1,2017-08-16,1,46.80,0.00,2,8,2017,1
2,2017-08-16,1,46.80,0.00,2,8,2017,2
3,2017-08-16,1,46.80,0.00,2,8,2017,3
4,2017-08-16,1,46.80,0.00,2,8,2017,4
...,...,...,...,...,...,...,...,...
28507,2017-08-31,9,47.26,0.00,3,8,2017,28
28508,2017-08-31,9,47.26,0.00,3,8,2017,29
28509,2017-08-31,9,47.26,0.00,3,8,2017,30
28510,2017-08-31,9,47.26,0.00,3,8,2017,31


In [29]:
xdf_test.iloc[151]

date          2017-08-16 00:00:00
store_nbr                      13
dcoilwtico                  46.80
locale                       0.00
day                             2
month                           8
year                         2017
family_int                     19
Name: 151, dtype: object

In [30]:
values = {
    'store_nbr'      :13.00,
    'date'           :'2017-08-16',
    'family_int'     :19.00
}

In [31]:
store = values['store_nbr']
family_int = values['family_int']
date = values['date']

temp = xdf_test[xdf_test['store_nbr']==store]
temp = temp[temp['family_int']==family_int]
temp = temp[temp['date']==date]
print(temp)
matching_index = temp.dropna().index[0]
matching_index

          date  store_nbr  dcoilwtico  locale  day  month  year  family_int
151 2017-08-16         13       46.80    0.00    2      8  2017          19


151

In [32]:
# def reg_predict(store, family_int, date):
    
#     temp = df[df['store_nbr']==store]
#     temp = temp[temp['family_int']==family_int]
#     temp.drop(columns = ['store_nbr', 'family_int','date'], inplace = True)
#     x_train = temp.drop(columns = ['sales','id'])
#     y_train = temp['sales']
    
#     regr_model = DecisionTreeRegressor(max_depth=8)
#     regr_model.fit(x_train, y_train)
    
#     temp = df_test[df_test['store_nbr']==store]
#     temp = temp[temp['family_int']==family_int]
#     temp.drop(columns = ['store_nbr', 'family_int'], inplace = True)
#     temp = temp[temp['date']==date]
 
#     x_test = temp.drop(columns=['id','date'])

#     y_test = regr_model.predict(x_test)
    
#     return y_test[0]
    

In [33]:
# values = {
#     'store_nbr'      :16.00,
#     'family_int'     :20.00
# }
# print(reg_predict(7,2,'2017-08-18'))

In [34]:
df

,id,date,store_nbr,sales,dcoilwtico,locale,day,month,year,family_int
0,0,2013-01-01,1,0.00,93.14,3.00,1,1,2013,0
1,1,2013-01-01,1,0.00,93.14,3.00,1,1,2013,1
2,2,2013-01-01,1,0.00,93.14,3.00,1,1,2013,2
3,3,2013-01-01,1,0.00,93.14,3.00,1,1,2013,3
4,4,2013-01-01,1,0.00,93.14,3.00,1,1,2013,4
...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,438.13,47.57,2.00,1,8,2017,28
3000884,3000884,2017-08-15,9,154.55,47.57,2.00,1,8,2017,29
3000885,3000885,2017-08-15,9,2419.73,47.57,2.00,1,8,2017,30
3000886,3000886,2017-08-15,9,121.00,47.57,2.00,1,8,2017,31


In [35]:
temp = df.copy()
temp.drop(columns = ['date','id'], inplace = True)
x_train = temp.drop(columns=['sales'])
y_train = temp['sales']

In [36]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer

# Define a custom RMSLE scorer function
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.log1p(y_pred) - np.log1p(y_true))**2)

rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

# Split the dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

# Define the hyperparameter grid for RandomizedSearchCV
param_dist = {
    'max_depth': [4, 6, 8, 10, 12],          # Example values for max_depth
    'min_samples_split': [2, 5, 10, 15],    # Example values for min_samples_split
    'min_samples_leaf': [1, 2, 4, 8],      # Example values for min_samples_leaf
}

# Create the DecisionTreeRegressor
regr_model = DecisionTreeRegressor()

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    regr_model, param_distributions=param_dist, n_iter=10, scoring=rmsle_scorer, cv=5, random_state=42, n_jobs=-1
)

# Perform the hyperparameter search
random_search.fit(x_train, y_train)

# Print the best hyperparameters and corresponding RMSLE score
print("Best Hyperparameters:", random_search.best_params_)
print("Best RMSLE Score:", -random_search.best_score_)

Best Hyperparameters: {'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 12}
Best RMSLE Score: 0.27633821268992115


In [37]:
regr_model = DecisionTreeRegressor(min_samples_split=10,min_samples_leaf=2,max_depth=12)
regr_model.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=12, min_samples_leaf=2, min_samples_split=10)

In [38]:
from sklearn.metrics import r2_score

In [39]:
y_pred = regr_model.predict(x_val)
print(r2_score(y_val,y_pred))

0.8967321881297614


In [40]:
def prediction(store, family_int, date):
    temp = df_test[df_test['store_nbr']==store]
    temp = temp[temp['family_int']==family_int]
    temp = temp[temp['date']==date]

    x_test = temp.drop(columns=['id','date'])

    y_test = regr_model.predict(x_test)

    return y_test[0]

In [41]:
print(prediction(1,0,'2017-08-16'))

4.088


In [42]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [43]:
steps = [
    ('scaler', StandardScaler()),  # Step 1: Data preprocessing
    ('classifier', regr_model)  # Step 2: Model
]

In [44]:
pipe = Pipeline(steps)

In [45]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 DecisionTreeRegressor(max_depth=12, min_samples_leaf=2,
                                       min_samples_split=10))])

In [46]:
y_pred = pipe.predict(x_val)
print(r2_score(y_val,y_pred))

0.8967307024257315


In [47]:
temp.to_csv('train_clean.csv', index=False)

In [48]:
df_test.head(1)

,id,date,store_nbr,dcoilwtico,locale,day,month,year,family_int
0,3000888,2017-08-16,1,46.80,0.00,2,8,2017,0


In [49]:
df_test['store_nbr'].dtype

dtype('int64')

In [50]:
df_test.to_csv('testdata.csv',index=False)

In [51]:
from joblib import dump

In [52]:
dump(pipe, "RegressorModel.joblib")

['RegressorModel.joblib']

In [53]:
import pickle

In [54]:
pickle.dump(pipe,open('RegressorModel.pkl','wb'))

In [55]:
len(family_list)

33

In [56]:
import sklearn
print(sklearn.__version__)

1.2.2
